In [1]:
from buffer import Buffer
from node import Node

In [2]:
from src.channel import Channel
from src.packet import Packet
from src.range import Range
TOTAL_PACKAGES = 50000

In [3]:
buffer_c = Buffer(capacity=TOTAL_PACKAGES)
node_c = Node(channels=[], buffer=buffer_c)

In [4]:
channel_b1c = Channel(range=Range(22, 28), receiver=node_c)
channel_b2c = Channel(range=Range(25, 25), receiver=node_c)

threshold_reached = False

def callback():
    global channel_b1c, channel_b2c, threshold_reached
    if not threshold_reached:
        channel_b1c.range = Range(15, 15)
        channel_b2c.range = Range(15, 15)
        threshold_reached = True

buffer_b = Buffer(capacity=25, threshold=20, on_threshold_reached=callback)

node_b1 = Node(channels=[channel_b1c], buffer=buffer_b)
node_b2 = Node(channels=[channel_b2c], buffer=buffer_b)

In [5]:
channel_ab1 = Channel(range=Range(20, 20), receiver=node_b1)
channel_ab2 = Channel(range=Range(15, 25), receiver=node_b2)

buffer_a = Buffer(capacity=20)

node_a = Node(channels=[channel_ab1, channel_ab2], buffer=buffer_a)

In [6]:
channel_emitter_a = Channel(range=Range(5, 15), receiver=node_a)

buffer_emitter = Buffer(capacity=TOTAL_PACKAGES)

packets_to_send = [Packet(i) for i in range(TOTAL_PACKAGES)]

for packet in packets_to_send:
    buffer_emitter.push(packet)

node_emitter = Node(channels=[channel_emitter_a], buffer=buffer_emitter)

In [7]:
time_aware_units = [
    node_emitter,
    channel_emitter_a,

    node_a,
    channel_ab1,
    channel_ab2,

    node_b1,
    channel_b1c,

    node_b2,
    channel_b2c,

    node_c
]

In [8]:
experiment_results = []


threshold_reached = False

for tick in range(TOTAL_PACKAGES * 50 + 1):
    for unit in time_aware_units:
        unit.on_clock_tick()

experiment_results.append((threshold_reached, TOTAL_PACKAGES - len(buffer_c)))

experiment_results

[(True, 119)]